# Init

In [105]:
# from skfem import *
import numpy as np
from utils import *
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
# from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time
from tqdm import tqdm


tol = 1e-8
intorder = 3 # 6
solver_type = 'mgcg'
refine_time = 7
element_type = 'P1'
sigma = 5 
penalty = False
epsilon = 1e-6
ep = epsilon
example = 'ex3'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

## Save $u_{h0}$ with different $h$ 

In [2]:
# # %%time
# for i in range(1, 10):
#     m = MeshTri()
#     base_order = i
#     base_path = 'uh0_{}.npy'.format(base_order)
#     m.refine(base_order)

#     if penalty:
#         uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
#     else:
#         uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

#     np.save(base_path, uh0)
#     print('{} th saved'.format(i))

## Reading previous solution

In [18]:
test_order = 2
test_path = 'uh0_{}.npy'.format(test_order)
m = MeshTri()
m.refine(test_order)
test_basis, test_fbasis = solve_problem2(m, element_type, intorder=3, basis_only=True)
test_uh0 = np.load(test_path)

In [76]:
base_order = 7
base_path = 'uh0_{}.npy'.format(base_order)
m = MeshTri()
m.refine(base_order)
base_basis, base_fbasis = solve_problem2(m, element_type, intorder=3, basis_only=True)
base_uh0 = np.load(base_path)

## L2 error

In [77]:
base_test_basis = np.zeros_like(base_basis['u'].interpolate(base_uh0).value)

In [78]:
coordinates = base_basis['u'].global_coordinates().value

In [79]:
# from numba import jit
# # @jit
# def test():
#     for i in tqdm(range(base_test_basis.shape[0])):
#         for j in range(base_test_basis.shape[1]):
#             base_test_basis[i][j] = test_basis['u'].interpolator(test_uh0)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))
#     return base_test_basis

In [80]:
for i in tqdm(range(base_test_basis.shape[0])):
    for j in range(base_test_basis.shape[1]):
        base_test_basis[i][j] = test_basis['u'].interpolator(test_uh0)(np.array([[coordinates[0][i][j]], [coordinates[1][i][j]]]))

100%|███████████████████████████████████████████████████████████████████████████| 32768/32768 [04:58<00:00, 109.71it/s]


In [92]:
base_test_basis

array([[-4.77772593e-03, -5.70887422e-03, -5.70887422e-03,
        -2.89103246e-03],
       [ 6.04554108e-04,  2.14659701e-03, -6.44463284e-04,
         3.46559789e-04],
       [ 6.04554108e-04, -6.44463284e-04,  2.14659701e-03,
         3.46559789e-04],
       ...,
       [ 2.08186866e-01,  2.09471769e-01,  2.09471769e-01,
         2.05592662e-01],
       [ 7.00363813e-01,  7.02813698e-01,  6.99116411e-01,
         6.99116411e-01],
       [ 7.00363813e-01,  6.99116411e-01,  6.99116411e-01,
         7.02813698e-01]])

In [103]:
np.sqrt(np.sum(base_basis['u'].dx * (base_basis['u'].interpolate(base_uh0).value - base_test_basis)**2))

0.07601233301088486

In [104]:
np.sqrt(np.sum(base_basis['u'].dx * (base_basis['u'].interpolate(base_uh0).value - exact_u(x, y))**2))

0.00036956008596858404

In [93]:
base_basis['u'].interpolate(base_uh0).value

array([[-6.69011090e-05, -4.81687985e-05, -4.81687985e-05,
        -7.22531977e-05],
       [ 6.69740234e-05,  4.82212968e-05,  2.16885809e-04,
         4.71544184e-08],
       [ 6.69740234e-05,  2.16885809e-04,  4.82212968e-05,
         4.71544222e-08],
       ...,
       [ 2.49979022e-01,  2.51373106e-01,  2.51373106e-01,
         2.47150598e-01],
       [ 7.49957012e-01,  7.52785517e-01,  7.48514712e-01,
         7.48514712e-01],
       [ 7.49957012e-01,  7.48514713e-01,  7.48514713e-01,
         7.52785517e-01]])

In [94]:
exact_u(x, y)

array([[6.69310452e-05, 7.22842386e-05, 7.22842386e-05, 2.40955203e-05],
       [1.33857611e-04, 9.63762755e-05, 2.89119537e-04, 4.81904601e-05],
       [1.33857611e-04, 2.89119537e-04, 9.63762755e-05, 4.81904601e-05],
       ...,
       [2.50000000e-01, 2.51394271e-01, 2.51394271e-01, 2.47171313e-01],
       [7.50000000e-01, 7.52828687e-01, 7.48557538e-01, 7.48557538e-01],
       [7.50000000e-01, 7.48557538e-01, 7.48557538e-01, 7.52828687e-01]])

In [97]:
@Functional
def L2uError(w):
    global x, y, www
    www = w
    x, y = w.x
    print(w.w)
    return (w.w - exact_u(x, y))**2

In [96]:
@Functional
def L2uError(w):
    x, y = w.x
    return (w.w - base_test_basis)**2

array([[6.69310452e-05, 7.22842386e-05, 7.22842386e-05, 2.40955203e-05],
       [1.33857611e-04, 9.63762755e-05, 2.89119537e-04, 4.81904601e-05],
       [1.33857611e-04, 2.89119537e-04, 9.63762755e-05, 4.81904601e-05],
       ...,
       [2.50000000e-01, 2.51394271e-01, 2.51394271e-01, 2.47171313e-01],
       [7.50000000e-01, 7.52828687e-01, 7.48557538e-01, 7.48557538e-01],
       [7.50000000e-01, 7.48557538e-01, 7.48557538e-01, 7.52828687e-01]])

In [89]:
base_test_basis

array([[-4.77772593e-03, -5.70887422e-03, -5.70887422e-03,
        -2.89103246e-03],
       [ 6.04554108e-04,  2.14659701e-03, -6.44463284e-04,
         3.46559789e-04],
       [ 6.04554108e-04, -6.44463284e-04,  2.14659701e-03,
         3.46559789e-04],
       ...,
       [ 2.08186866e-01,  2.09471769e-01,  2.09471769e-01,
         2.05592662e-01],
       [ 7.00363813e-01,  7.02813698e-01,  6.99116411e-01,
         6.99116411e-01],
       [ 7.00363813e-01,  6.99116411e-01,  6.99116411e-01,
         7.02813698e-01]])

In [47]:
www.x.shape

(2, 512, 4)

In [49]:
www.w

array([ 0.        ,  0.        ,  0.        , ...,  1.83138579,
        1.5301719 , -1.5301719 ])

In [43]:
base_test_basis.shape

(512, 4)

In [42]:
x.shape

(512, 4)

In [98]:
np.sqrt(L2uError.assemble(base_basis['u'], w=base_basis['u'].interpolate(base_uh0)))

[[-6.69011090e-05 -4.81687985e-05 -4.81687985e-05 -7.22531977e-05]
 [ 6.69740234e-05  4.82212968e-05  2.16885809e-04  4.71544184e-08]
 [ 6.69740234e-05  2.16885809e-04  4.82212968e-05  4.71544222e-08]
 ...
 [ 2.49979022e-01  2.51373106e-01  2.51373106e-01  2.47150598e-01]
 [ 7.49957012e-01  7.52785517e-01  7.48514712e-01  7.48514712e-01]
 [ 7.49957012e-01  7.48514713e-01  7.48514713e-01  7.52785517e-01]]


0.00036956008596858404

In [87]:
np.sqrt(L2uError.assemble(base_basis['u'], w=base_basis['u'].interpolate(base_uh0)))

0.07601233301088486

In [ ]:
m = MeshTri()
m.refine(3)
base_uh0, base_basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
m = MeshTri()
m.refine(5)
test_uh0, test_basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

## projection

In [ ]:
# uh0_project = project(test_uh0, basis_from=test_basis['u'], basis_to=base_basis['u'])

In [ ]:
basis_to = InteriorBasis(m, ElementTriMorley(), intorder=3)
# test_basis['u']

In [ ]:
uh0_project = project(test_uh0, basis_to, base_basis['u'])

In [ ]:
basis_store['u'].interpolator(uh0_store)(np.array([[x[i][j]], [y[i][j]]]))

In [110]:
m = MeshTri()
m.refine(7)
uh0, basis = solve_problem1(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)

In [111]:
# compute errors
U = basis['u'].interpolate(uh0).value

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))

In [112]:
L2u

0.00039684038351778554

In [ ]:
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

In [ ]:
uh0_store = uh0
basis_store = basis

In [ ]:
%%time
m = MeshTri()
m.refine(3)

if penalty:
    uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
else:
    uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

U = basis['u'].interpolate(uh0).value

# compute errors

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

In [ ]:
L2u

In [ ]:
def exact_u_n(x, y):
    return basis_store['u'].interpolator(uh0_store)(np.array([[x], [y]]))

In [ ]:
@Functional
def L2uError_h(w):
    x, y = w.x
    return (w.w - un)**2

In [ ]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

In [ ]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

In [ ]:
x, y = www.x
exact_u(x, y)

In [ ]:
un = np.zeros_like(x)
for i in tqdm(range(x.shape[0])):
    for j in range(x.shape[1]):
        un[i][j] = basis_store['u'].interpolator(uh0_store)(np.array([[x[i][j]], [y[i][j]]]))

In [ ]:
uh0.shape

In [ ]:
un

In [ ]:
U.shape

In [ ]:
uh0.shape

In [ ]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

In [ ]:
m = MeshTri()
# m.refine(1)

if element_type == 'P1':
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
elif element_type == 'P2':
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
else:
    raise Exception("The element not supported")

basis = {
    variable: InteriorBasis(m, e, intorder=3)
    for variable, e in element.items()
}

In [ ]:
# basis['u'].dx # weights
refine2 = basis['u'].global_coordinates().value

In [ ]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

# Solving

In [113]:
# solving 

print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('epsilon:\t{}'.format(epsilon))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

time_start = time.time()

ep = epsilon
df_list = []
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, refine_time+1):

    m.refine()

    if penalty:
        uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=intorder, tol=1e-8, epsilon=1e-6)
    else:
        uh0, basis = solve_problem1(m, element_type, solver_type, intorder=intorder, tol=1e-8, epsilon=1e-6)

    U = basis['u'].interpolate(uh0).value

    # compute errors

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    
    if penalty:
        D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
    else:
        D2u = get_D2uError(basis['u'], uh0)
        
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us

# store data
data = np.array([L2s, H1s, H2s, epus])
df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
df_list.append(df)

show_result(L2s, H1s, H2s, epus)

# df.to_csv(save_path+'.csv')
time_end = time.time()
print('Total Time Cost {:.2f} s'.format(time_end-time_start))

=======Arguments=======
penalty:	False
element_type:	P1
solver_type:	mgcg
tol:	1e-08
intorder:	3
refine_time:	7
epsilon:	1e-06
sigma:	5
=======Results=======
  h    L2u   H1u   H2u   epu
2^-2  1.74  0.81  -0.19  0.70
2^-2  7.282e-02  1.058e+00  1.310e+01  9.853e-01
2^-3  1.59  0.65  -0.46  0.60
2^-3  2.421e-02  6.761e-01  1.807e+01  6.519e-01
2^-4  1.52  0.55  -0.49  0.52
2^-4  8.455e-03  4.632e-01  2.543e+01  4.547e-01
2^-5  1.51  0.52  -0.49  0.50
2^-5  2.973e-03  3.238e-01  3.580e+01  3.208e-01
2^-6  1.51  0.51  -0.50  0.50
2^-6  1.047e-03  2.278e-01  5.047e+01  2.267e-01
2^-7  1.50  0.50  -0.50  0.50
2^-7  3.696e-04  1.607e-01  7.124e+01  1.603e-01
Total Time Cost 4.55 s
